In [1]:
import os, glob
import pandas as pd
import numpy as np
import pybedtools 
from Bio import SeqIO
from io import StringIO
pybedtools.helpers.set_tempdir("/data/projects/temp")
from tqdm import tqdm

In [2]:
non_coding_region= "donor"

In [3]:
output_path = "/data/projects/DNABERT_snv/Manuscript_11_2023/Intersected_data/dbsnp/{}".format(non_coding_region)
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [4]:
df_donor = pd.read_csv("/data/projects/DNABERT_snv/Manuscript_11_2023/bed/DNABERT_run_data/donor.bed", sep="\t", header=None)
df_donor[2] = df_donor[2].round().astype(int)
df_donor

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,chr7,127589124,127589203,ENST00000000233,ENSG00000004059,+,127588411,1032,protein_coding,56.14,ARF5,HouseTrans,"Adipose_Tissue, Adrenal_Gland, Bladder, Blood,..."
1,chr7,127588526,127588605,ENST00000000233,ENSG00000004059,+,127588411,1032,protein_coding,56.14,ARF5,HouseTrans,"Adipose_Tissue, Adrenal_Gland, Bladder, Blood,..."
2,chr7,127589555,127589634,ENST00000000233,ENSG00000004059,+,127588411,1032,protein_coding,56.14,ARF5,HouseTrans,"Adipose_Tissue, Adrenal_Gland, Bladder, Blood,..."
3,chr7,127591049,127591128,ENST00000000233,ENSG00000004059,+,127588411,1032,protein_coding,56.14,ARF5,HouseTrans,"Adipose_Tissue, Adrenal_Gland, Bladder, Blood,..."
4,chr7,127591661,127591740,ENST00000000233,ENSG00000004059,+,127588411,1032,protein_coding,56.14,ARF5,HouseTrans,"Adipose_Tissue, Adrenal_Gland, Bladder, Blood,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175762,chr17,63706595,63706674,ENST00000640999,ENSG00000266173,-,63741796,2336,protein_coding,48.88,STRADA,NullTrans,NaN
1175763,chr17,63707207,63707286,ENST00000640999,ENSG00000266173,-,63741796,2336,protein_coding,48.88,STRADA,NullTrans,NaN
1175764,chr17,63710451,63710530,ENST00000640999,ENSG00000266173,-,63741796,2336,protein_coding,48.88,STRADA,NullTrans,NaN
1175765,chr17,63710688,63710767,ENST00000640999,ENSG00000266173,-,63741796,2336,protein_coding,48.88,STRADA,NullTrans,NaN


In [5]:
df_donor[[0, 1, 2]].drop_duplicates()

,0,1,2
0,chr7,127589124,127589203
1,chr7,127588526,127588605
2,chr7,127589555,127589634
3,chr7,127591049,127591128
4,chr7,127591661,127591740
...,...,...,...
1175709,chr4,82320446,82320525
1175710,chr4,82339974,82340053
1175719,chr5,162154973,162155052
1175736,chr6,160491582,160491661


In [8]:
donor_bed = pybedtools.BedTool.from_dataframe(df_donor)

In [9]:
chr_wise_DBSNP_files = glob.glob('/data/projects/VCF_files/DBSNP/ChrWise_Data/*.csv')

In [10]:
for dbsnp_file in chr_wise_DBSNP_files:
    # Read the csv file into a DataFrame
    print(dbsnp_file)
    df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])
    df = df.iloc[1:]
    print(df.head())
    
    # Convert the DataFrame to a BedTool
    dbsnp = pybedtools.BedTool.from_dataframe(df)
    
    # Intersect the BedTool with the transcript file
    intersected = donor_bed.intersect(dbsnp, wa=True, wb=True)
    
    # Save the intersected BedTool to a bed file
    output_file =output_path +  "/Chrwise/intersected_DBSNP_" + dbsnp_file.split('/')[-1].replace('.csv', '.tsv')
    if not os.path.exists(os.path.dirname(output_file)):
        os.makedirs(os.path.dirname(output_file))
    intersected.saveas(output_file)

/data/projects/VCF_files/DBSNP/ChrWise_Data/chr7_data.csv


/data/projects/temp/ipykernel_1814377/1276204644.py:4: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])


      0      1      2             3  4  5   6
1  chr7  10001  10002  rs1583156258  T  1  C,
2  chr7  10002  10003  rs1583156262  A  1  G,
3  chr7  10003  10004  rs1583156265  A  1  C,
4  chr7  10004  10005  rs1583156268  C  1  A,
5  chr7  10006  10007  rs1781584841  C  1  T,
/data/projects/VCF_files/DBSNP/ChrWise_Data/chr13_data.csv


/data/projects/temp/ipykernel_1814377/1276204644.py:4: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])


       0         1         2             3  4  5   6
1  chr13  16000025  16000026  rs1175737345  T  1  A,
2  chr13  16000063  16000064  rs1957610123  G  1  A,
3  chr13  16000079  16000080  rs1957610132  G  1  T,
4  chr13  16000081  16000082  rs1957610136  A  1  C,
5  chr13  16000100  16000101  rs1957610144  T  1  C,
/data/projects/VCF_files/DBSNP/ChrWise_Data/chrY_data.csv


...

      0      1      2             3   4  5       6
1  chr3  10008  10009  rs1339898020   A  3  C,G,T,
2  chr3  10009  10010  rs1695495101   A  1      C,
3  chr3  10010  10011  rs1695495136   C  1      A,
4  chr3  10014  10015  rs1206436362   A  1      C,
5  chr3  10014  10016  rs1695495167  AA  1    AAA,
/data/projects/VCF_files/DBSNP/ChrWise_Data/chr17_data.csv


/data/projects/temp/ipykernel_1814377/1276204644.py:4: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])


       0      1      2             3  4  5   6
1  chr17  60054  60055  rs2039333033  G  1  A,
2  chr17  60086  60087  rs2039333047  C  1  T,
3  chr17  60102  60103  rs1399099657  G  1  A,
4  chr17  60103  60104  rs1363626035  C  1  T,
5  chr17  60149  60150  rs2039333080  C  1  A,
/data/projects/VCF_files/DBSNP/ChrWise_Data/chr2_data.csv


/data/projects/temp/ipykernel_1814377/1276204644.py:4: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])


      0      1      2             3        4  5         6
1  chr2  10006  10007  rs1572047073        C  1        A,
2  chr2  10015  10015  rs1558169263      NaN  1        G,
3  chr2  10016  10023  rs1558169385  CACCCAC  1      CAC,
4  chr2  10018  10021  rs1558169386      CCC  1       CC,
5  chr2  10019  10019  rs1558169388      NaN  2  ACA,ACG,


In [11]:
chr_wise_path = os.path.dirname(output_file)
tsv_files = [f for f in os.listdir(chr_wise_path) if f.endswith('.tsv') and "chrM" not in f]

In [12]:
list_of_dfs = []

for file in tsv_files:
    file_path = os.path.join(chr_wise_path , file)
    try:
        # Attempt to read the .tsv file
        df = pd.read_csv(file_path, sep='\t', header=None)
        print(df.shape)
        list_of_dfs.append(df)
    except pd.errors.EmptyDataError:
        print(f"Warning: {file_path} is empty and will be skipped.")

(2379237, 20)
(1655330, 20)
(1447059, 20)
(2200068, 20)
(444686, 20)
(857983, 20)
(1817161, 20)
(1349332, 20)
(831431, 20)
(1603349, 20)
(2993249, 20)
(1400434, 20)
(564350, 20)
(2527158, 20)
(2083388, 20)
(2561930, 20)
(1457186, 20)
(683317, 20)
(1355651, 20)
(898862, 20)
(1351303, 20)
(12189, 20)
(2212606, 20)
(2358210, 20)


In [15]:
# Concatenate all the DataFrames in the list into a single DataFrame
final_df = pd.concat(list_of_dfs, ignore_index=True) if list_of_dfs else pd.DataFrame()
#final_df=final_df.drop(columns=[3])
new_columns = ['chr_name', '{}_start'.format(non_coding_region), '{}_end'.format(non_coding_region), "ENSEMBL_Transcript_ID","ENSEMBL_Gene ID", "strand","TSS","Transcript_length","Transcript_type","Gene_CG_content","GENE_symbol","Category","Tissue", 'chromosome', 'Variant_start', 'Variant_end', 'RS_ID', 'Reference_Nucleotide', 'No_of_variations', 'Alternative_Nucleotides']
final_df.columns = new_columns
final_df

,chr_name,donor_start,donor_end,ENSEMBL_Transcript_ID,ENSEMBL_Gene ID,strand,TSS,Transcript_length,Transcript_type,Gene_CG_content,GENE_symbol,Category,Tissue,chromosome,Variant_start,Variant_end,RS_ID,Reference_Nucleotide,No_of_variations,Alternative_Nucleotides
0,chr11,64315231,64315310,ENST00000000442,ENSG00000173153,+,64305524,2274,protein_coding,57.13,ESRRA,HouseTrans,"Adipose_Tissue, Adrenal_Gland, Bladder, Blood,...",chr11,64315233,64315245,rs1484286606,TATGTTCTACTA,1,"TA,"
1,chr11,64315231,64315310,ENST00000000442,ENSG00000173153,+,64305524,2274,protein_coding,57.13,ESRRA,HouseTrans,"Adipose_Tissue, Adrenal_Gland, Bladder, Blood,...",chr11,64315250,64315251,rs750270682,C,1,"T,"
2,chr11,64315231,64315310,ENST00000000442,ENSG00000173153,+,64305524,2274,protein_coding,57.13,ESRRA,HouseTrans,"Adipose_Tissue, Adrenal_Gland, Bladder, Blood,...",chr11,64315259,64315260,rs2035223711,T,1,"C,"
3,chr11,64315231,64315310,ENST00000000442,ENSG00000173153,+,64305524,2274,protein_coding,57.13,ESRRA,HouseTrans,"Adipose_Tissue, Adrenal_Gland, Bladder, Blood,...",chr11,64315260,64315261,rs2035223790,G,1,"A,"
4,chr11,64315231,64315310,ENST00000000442,ENSG00000173153,+,64305524,2274,protein_coding,57.13,ESRRA,HouseTrans,"Adipose_Tissue, Adrenal_Gland, Bladder, Blood,...",chr11,64315268,64315269,rs1361791105,A,1,"G,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37045464,chr2,85839156,85839235,ENST00000640992,ENSG00000115525,-,85888776,2292,protein_coding,45.07,ST3GAL5,NullTrans,.,chr2,85839224,85839225,rs1487482349,A,1,"G,"
37045465,chr2,85839156,85839235,ENST00000640992,ENSG00000115525,-,85888776,2292,protein_coding,45.07,ST3GAL5,NullTrans,.,chr2,85839225,85839226,rs3202465,G,1,"A,"
37045466,chr2,85839156,85839235,ENST00000640992,ENSG00000115525,-,85888776,2292,protein_coding,45.07,ST3GAL5,NullTrans,.,chr2,85839226,85839227,rs115591304,A,1,"G,"
37045467,chr2,85839156,85839235,ENST00000640992,ENSG00000115525,-,85888776,2292,protein_coding,45.07,ST3GAL5,NullTrans,.,chr2,85839227,85839228,rs1267804946,A,1,"G,"


In [19]:
# Write the final DataFrame to a new .tsv file (only if there's data)
if not final_df.empty:
    output_intersected_file_path = output_path+"/all_{}_intesected_data.tsv".format(non_coding_region)
    final_df.to_csv(output_intersected_file_path, sep='\t', index=False)
    print(f"All .tsv files (excluding chrM) concatenated into {output_intersected_file_path}")
else:
    print("No data to save after excluding empty files and chrM.")

All .tsv files (excluding chrM) concatenated into /data/projects/DNABERT_snv/Manuscript_11_2023/Intersected_data/dbsnp/donor/all_donor_intesected_data.tsv
